### Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

### Importing the Training & Test Sets

In [ ]:
training_set = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-3/train.csv', index_col = 'Id')
test_set = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-3/test.csv', index_col = 'ForecastId')

In [ ]:
training_set.head()

In [ ]:
test_set.head()

### Dataset description

In [ ]:
training_set.describe()

In [ ]:
test_set.describe()

### Creating copies of the datasets

In [ ]:
training_set_2 = training_set.copy()
test_set_2 = test_set.copy()

### Extracting the 'Fatalities' & 'ConfirmedCases' columns as Numpy Arrays

In [ ]:
train_confirmedcases = np.array(training_set['ConfirmedCases'].astype(int))
train_fatalities = np.array(training_set['Fatalities'].astype(int))

In [ ]:
train_confirmedcases[0:5]

In [ ]:
train_fatalities[0:5]

### Concatenating Training & Test Set

In [ ]:
dataset = pd.concat([training_set.drop(['ConfirmedCases', 'Fatalities'], axis=1), test_set])
dataset.count()

### Replacing the missing values in Province_State with No_Data

In [ ]:
dataset['Province_State'].fillna('No_Data', inplace = True)
dataset.head()

### Converting the Categorical Variables including the Dates to Dummy Variables

In [ ]:
dataset = pd.get_dummies(dataset, columns = dataset.columns)
dataset.head()

### Splitting the Processed Training & Test Set

In [ ]:
split = training_set.shape[0]
X_train = dataset[:split]
X_test= dataset[split:]

In [ ]:
X_train.count()

In [ ]:
X_test.count()

### Finding the Test Predictions using LightGBM Regression models

In [ ]:
regressor = LGBMRegressor(boosting_type = 'dart', learning_rate= 0.001, random_state = 0, n_jobs = -1)
regressor.fit(X_train.values, train_confirmedcases)
predicted_confirmedcases = regressor.predict(X_test.values)
predicted_confirmedcases

In [ ]:
regressor_2 = LGBMRegressor(boosting_type = 'dart', learning_rate= 0.0001, num_leaves = 124, random_state = 0, n_jobs = -1)
regressor_2.fit(X_train.values, train_fatalities)
predicted_fatalities = regressor_2.predict(X_test.values)
predicted_fatalities

### Exporting the Submission CSV

In [ ]:
submission = pd.DataFrame({'ForecastId': test_set.index,'ConfirmedCases': predicted_confirmedcases,'Fatalities': predicted_fatalities})
submission.to_csv('submission.csv',index = False)